## Microwave Oven Simulation

### Microwave Oven Model

In [1]:
import numpy as np

dx = 5
# model size
model_height = 450
model_width = 250
model_depth = 330

# box size
box_height = 450
box_width = 250
box_depth = 360

# inner size
inner_height = 350
inner_width = 200
inner_depth = 300

# door size
door_height = 450
door_width = 250
door_depth = 30

# mesh size
mesh_height = 320
mesh_width = 180
mesh_depth = 10

oh = int(model_height / dx) + 50
ow = int(model_width / dx)  + 50
od = int(model_depth / dx)  + 100

bh = int(box_height / dx)
bw = int(box_width / dx)
bd = int(box_depth / dx)

ih = int(inner_height / dx)
iw = int(inner_width / dx)
id = int(inner_depth / dx)

mh = int(mesh_height / dx)
mw = int(mesh_width / dx)
md = int(mesh_depth / dx)

dh = int(door_height / dx)
dw = int(door_width / dx)
dd = int(door_depth / dx)

ob_h = (oh - bh) // 2
ob_w = (ow - bw) // 2
ob_d = (od - bd) // 2

bi_h = (bh - ih) // 2
bi_w = (bw - iw) // 2
bi_d = (bd - id) // 2

dw_h = (dh - mh) // 2
dw_w = (dw - mw) // 2

bd_h = (bh - dh) // 2
bd_w = (bw - dw) // 2
bd_d = (bd - dd) // 2

# model
print('=== size ===')
model  = np.full((oh, ow, od),0,dtype=np.int8)
# Create box
box    = np.full((bh, bw, bd),1,dtype=np.int8)
# Create inner
inner  = np.full((ih, iw, id),0,dtype=np.int8)
# Create mesh
mesh   = np.full((mh, mw, md),2,dtype=np.int8)
# Create door
door   = np.full((dh, dw, dd),3,dtype=np.int8)
# Create window
window = np.full((mh, mw, dd),4,dtype=np.int8)

print(f'name   : X , Y , Z ')
print(f'model  :{oh:3d},{ow:3d},{od:3d}')
print(f'box    :{bh:3d},{bw:3d},{bd:3d}')
print(f'inner  :{ih:3d},{iw:3d},{id:3d}')
print(f'mesh   :{mh:3d},{mw:3d},{md:3d}')
print(f'door   :{dh:3d},{dw:3d},{dd:3d}')
print(f'window :{mh:3d},{mw:3d},{dd:3d}')

# Set mesh hole
for i in range(2, mh, 4):
    for j in range(2, mw, 4):
        mesh[i:i+1,j:j+1,:] = 0

# Create microwave oven model
window[:,:,dd//2-md//2:dd//2+md//2] = mesh
door[dw_h:-dw_h,dw_w:-dw_w,:] = window
box[bi_h:-bi_h, bi_w:-bi_w, dd:dd+id] = inner
box[:, :, 0:dd]= door
model[ob_h:-ob_h, ob_w:-ob_w, ob_d:-ob_d] = box

# save model
np.save('out/model.npy',model)

=== size ===
name   : X , Y , Z 
model  :140,100,166
box    : 90, 50, 72
inner  : 70, 40, 60
mesh   : 64, 36,  2
door   : 90, 50,  6
window : 64, 36,  6


### Model Visualization

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

model = np.load('out/model.npy')
colors = [(0, 0, 0.5), (0, 1, 0), (1, 0, 0), (1, 1, 0)] # blue, green, red, yellow
custom_map = mcolors.ListedColormap(colors)

x,y,z = model.shape

fig,axs = plt.subplots(2,2)

axs[0,0].set_title('X')
axs[0,0].set_xlabel('Z')
axs[0,0].set_ylabel('Y')
axs[0,0].imshow(model[x//2,:,:],interpolation='nearest',cmap=custom_map)

axs[0,1].set_title('Y')
axs[0,1].set_xlabel('Z')
axs[0,1].set_ylabel('X')
axs[0,1].imshow(model[:,y//2,:],interpolation='nearest',cmap=custom_map)

axs[1,0].set_title('Z')
axs[1,0].set_xlabel('X')
axs[1,0].set_ylabel('Y')
axs[1,0].imshow(model[:,:,z//2].T,interpolation='nearest',cmap=custom_map)

axs[1,1].set_title('shield')
axs[1,1].set_xlabel('X')
axs[1,1].set_ylabel('Y')
axs[1,1].imshow(model[:,:,50].T,interpolation='nearest',cmap=custom_map)

fig.tight_layout()
fig.savefig('out/4_1.jpg')
plt.close(fig)

<img src="out/4_1.jpg"> 

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker

# fdtd
import fdtd

vacuum    = [1, 0]       
aluminum  = [7.81, 3.96e7]
glass     = [3.7,1e-12]
constants = [vacuum,aluminum,aluminum,aluminum,glass] # List[[vacuum],[box],[mesh],[door],[window]]

f     = 2.45e9  # inp freq 　2.45Ghz
ddx   = 0.001   # cell size 1mm * 1mm * 1mm

model = np.load('out/model.npy')
grid  = fdtd.fdtd(model,constants,ddx,f,DTYPE=np.float32)
grid.set_boundaries('PML')

# run
ims = []
fig,ax = plt.subplots()
NT:int = 300
colors = [(0, 0, 1), (0, 1, 0), (1, 0, 0), (1, 1, 0)] # blue, green, red, yellow
custom_map = mcolors.ListedColormap(colors)

for n in range(NT):    
    data = grid.calculate_amp(grid.dx,grid.dy,grid.dz)
    im1 = ax.imshow(data[grid.nxc,:,:], cmap='jet', origin='lower',vmin=0, vmax=2e-5)
    im2 = ax.imshow(np.ma.masked_where(model[grid.nxc] == 0, model[grid.nxc]),interpolation='nearest',cmap=custom_map,alpha=0.6)
    ims.append([im1,im2])
    grid.run()

cbar = fig.colorbar(im1,shrink=0.65)
cbar.ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
plt.close()

plt.rcParams['animation.embed_limit'] = 50
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=False)
ani.save("out/4_1.gif", writer="imagemagick")

<img src="out/4_1.gif">